In [1]:
from recorder import MASDataset, SingleQuestion, SingleResponse
import re
import numpy as np
from sklearn.metrics import roc_auc_score

dataset = MASDataset.load_pickle("/home/hanwen/workspace/2025/MASLab/results/Meta-Llama-3-8B-Instruct/MMLU_500/llm_debate_infer_semantic_entropy.pkl")

pattern = r'[A-F]\)'
valid_questions = []
correct_list = []

for question in dataset.questions:
    try:
        final_answer = '(' + list(re.findall(pattern, question.final_response.generation_list[0]))[-1]
        single_answer = '(' + list(re.findall(pattern, question.responses[0][0].generation_list[0]))[-1]
        print(final_answer, single_answer, question.gt_answer['gt'], question.gt_answer['gt']==single_answer)
        question.set_correct(final_answer == question.gt_answer['gt'])
        correct_list.append(question.gt_answer['gt']==single_answer)
        valid_questions.append(question)
    except Exception as e:
        pass


(B) (B) (B) True
(D) (D) (D) True
(B) (B) (A) False
(C) (C) (C) True
(B) (B) (B) True
(D) (D) (D) True
(B) (B) (B) True
(B) (B) (B) True
(C) (C) (C) True
(C) (C) (D) False
(B) (B) (B) True
(D) (D) (D) True
(A) (A) (A) True
(C) (B) (C) False
(A) (C) (B) False
(C) (C) (C) True
(C) (C) (A) False
(C) (C) (C) True
(C) (C) (C) True
(C) (C) (B) False
(B) (A) (A) True
(C) (B) (A) False
(D) (D) (A) False
(B) (B) (B) True
(A) (A) (A) True
(B) (B) (B) True
(A) (A) (B) False
(D) (D) (D) True
(B) (D) (B) False
(A) (A) (A) True
(D) (A) (A) True
(B) (B) (A) False
(A) (A) (A) True
(C) (C) (C) True
(C) (C) (C) True
(C) (C) (C) True
(D) (D) (D) True
(D) (D) (D) True
(D) (D) (A) False
(A) (A) (A) True
(A) (D) (B) False
(B) (B) (A) False
(A) (A) (A) True
(D) (D) (D) True
(C) (D) (D) True
(B) (B) (B) True
(B) (B) (B) True
(A) (A) (A) True
(A) (A) (A) True
(A) (A) (A) True
(D) (D) (D) True
(D) (D) (D) True
(C) (C) (C) True
(C) (C) (D) False
(B) (B) (B) True
(B) (D) (B) False
(B) (B) (B) True
(A) (A) (A) Tru

In [2]:
print(np.mean(correct_list))

0.65


In [7]:
question = dataset.questions[0]
print(question.question)

The following is a multiple-choice question:
This question refers to the following information.
"Al-Zawawi also said 'This sultan Musa told me that at a town called ZKRY he has a copper mine from which ingots are brought to BYTY. "There is nothing in my kingdom, Musa said, on which a duty is levied except this crude copper which is brought in. Duty is collected on this and on nothing else. We send it to the land of the pagan Sudan and sell it for two-thirds of its weight in gold, so that we sell 100 measures of this copper for 66 2/3 measures of gold.' He also stated that there are pagan nations in his kingdom from whom he does not collect the tribute (jizya) but whom he simply employs in extracting the gold from its deposits. The gold is extracted by digging pits about a man's height in depth and the gold is found embedded in the sides of the pits or sometimes collected at the bottom of them."
From N. Levtzion & J.F.P. Hopkins, eds. Corpus of Early
Arabic Sources for West African Hist

In [8]:
print(question.responses[0][0].generation_list[0])

Let's analyze the passage to determine the most supported inference.

The passage describes the Sultan Musa's kingdom, which is implied to be Mali, and its trade in copper and gold. The Sultan collects a duty on the crude copper brought into his kingdom, which is then sold for gold in the land of the pagan Sudan. The passage also mentions that the Sultan employs pagan nations in his kingdom to extract gold from its deposits.

Now, let's evaluate the options:

(A) Mali was able to forge advanced bronze tools from its mineral resources.

The passage does not mention the use of copper to forge bronze tools. In fact, it only mentions the sale of copper for gold, without any reference to its use in tool-making.

(B) International trade with Sub-Saharan Africa involved the exchange of mineral resources.

This option is supported by the passage, which describes the trade in copper and gold between the Sultan's kingdom (Mali) and the land of the pagan Sudan.

(C) The jizya could only be collec

In [ ]:
print(question.gt_answer['gt'])

In [3]:
correctness = [question.correct for question in valid_questions]
print(np.mean(correctness))
print(len(correctness))

0.6595238095238095
420


In [ ]:
uncertainty_list = []
for question in valid_questions:
    uncertainty_multi_agent = []
    for round_num in question.responses:
        for agent_id in question.responses[round_num]:
            single_response = question.responses[round_num][agent_id]
            if single_response.uncertainty is not None:
                uncertainty_multi_agent.append(single_response.uncertainty['semantic_entropy'])
                print(single_response.uncertainty['semantic_ids'])
    uncertainty_list.append(np.max(uncertainty_multi_agent))
            

In [ ]:
single_response.uncertainty.keys()

In [ ]:
auroc = roc_auc_score(correctness, -np.array(uncertainty_list))
print(f"AUROC: {auroc:.4f}")